# Searching and exploring Ontologies types from the Blue Brain Knowledge Graph using the Knowledge Graph Forge

## Initialize and configure

### Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


Once a token is obtained then proceed to paste it below.

In [1]:
import getpass

In [ ]:
TOKEN = getpass.getpass()

### Configure a client (forge) to access the knowledge graph 

In [3]:
from kgforge.core import KnowledgeGraphForge

In [4]:
# Let target a project in the KG
ORG = "neurosciencegraph"
PROJECT = "datamodels"

In [ ]:
forge = KnowledgeGraphForge("prod-forge-nexus.yml", bucket=f"{ORG}/{PROJECT}", token=TOKEN)

## Show all ontologies

In [ ]:
query = {"type":"Ontology"}
ontologies = forge.search(query, limit=100, search_in_graph=False, cross_bucket=True)
forge.as_dataframe(ontologies)

## Retrieve a specific ontology

In [8]:
ontology_id = "http://bbp.epfl.ch/neurosciencegraph/ontologies/mba" #https://bbp.epfl.ch/ontologies/core/bmo
ontology = forge.retrieve(id=ontology_id, cross_bucket=True)

In [9]:
forge.as_dataframe(ontology)

,id,type,label,distribution,name,wasDerivedFrom.id
0,http://bbp.epfl.ch/neurosciencegraph/ontologie...,"[Ontology, Entity]",AIBS Mouse CCF Atlas Ontology,"[{'type': 'DataDownload', 'contentSize': {'uni...",AIBS Mouse CCF Atlas Ontology,http://ontology.neuinfo.org/NIF/ttl/generated/...


## Download a given ontology

In [10]:
dirpath = "./downloaded/"
forge.download(ontology, "distribution.contentUrl", dirpath)

## Export types defined in a given ontology along with their properties
Useful for local term (label, synonym, ...) to ontology type identifier mapping

In [11]:
query = {"isDefinedBy":{"id":f"{ontology_id}"}}
defined_types = forge.search(query, limit=2000, search_in_graph=False)

In [ ]:
defined_types_df = forge.as_dataframe(defined_types)
defined_types_df

In [13]:
dirpath = "./downloaded/ontology_types.csv"
defined_types_df.to_csv(dirpath)

## Retrieve an ontology type by name

In [14]:
type_ = "BrainRegion"
type_uri = forge._model.context().expand(type_)

In [15]:
# Yield a Resource => https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource
type_resource = forge.retrieve(id=type_uri, cross_bucket=True) 
print(type_resource)

{
    context: https://neuroshapes.org
    id: https://neuroshapes.org/BrainRegion
    type: Class
    label: Brain Region
    altLabel: regional part of the brain
    definition: Anatomical divisons of the brain according to one or more criteria, e.g. cytoarchitectural, gross anatomy. Parts may be contiguous in 3D or not, e.g., basal ganglia.
    isDefinedBy: https://bbp.epfl.ch/ontologies/core/bmo
    subClassOf: bmo:BiologicalBrainComponent
}


In [16]:
forge.as_json(type_resource) # A Resource can be accessed as JSON

{'id': 'https://neuroshapes.org/BrainRegion',
 'type': 'Class',
 'label': 'Brain Region',
 'altLabel': 'regional part of the brain',
 'definition': 'Anatomical divisons of the brain according to one or more criteria, e.g. cytoarchitectural, gross anatomy. Parts may be contiguous in 3D or not, e.g., basal ganglia.',
 'isDefinedBy': 'https://bbp.epfl.ch/ontologies/core/bmo',
 'subClassOf': 'bmo:BiologicalBrainComponent'}

## Get the identifier of the ontology that defines an ontology type

In [17]:
# Resource properties can be direcly accessed. Autocompletion on the properties is enabled
type_resource.isDefinedBy 

'https://bbp.epfl.ch/ontologies/core/bmo'

## See all (transitive) subclasses of a type

In [18]:
query = {"subClassOf*":{"id":type_}} # Remove the '*' for direct subclasses.

subclasses = forge.search(query, limit=10, search_in_graph=False, cross_bucket=True)

In [20]:
forge.as_dataframe(subclasses)

,id,type,label,atlas_id,color_hex_triplet,graph_order,hasPart,hemisphere_id,identifier,isDefinedBy,isPartOf,notation,prefLabel,st_level,subClassOf
0,http://api.brain-map.org/api/v2/data/Structure...,Class,Anteromedial visual area,756,08858C,171,"[mba:281, mba:1066, mba:433, mba:401, mba:1046...",3,394,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:669,VISam,Anteromedial visual area,8,nsg:BrainRegion
1,http://api.brain-map.org/api/v2/data/Structure...,Class,"Primary somatosensory area, barrel field, layer 4",979,188064,54,NaN,3,1047,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:329,SSp-bfd4,"Primary somatosensory area, barrel field, layer 4",11,nsg:BrainRegion
2,http://api.brain-map.org/api/v2/data/Structure...,Class,Gigantocellular reticular nucleus,130,FFB3D9,975,NaN,3,1048,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:370,GRN,Gigantocellular reticular nucleus,8,nsg:BrainRegion
3,http://api.brain-map.org/api/v2/data/Structure...,Class,"Medulla, motor related",187,FFB3D9,964,"[mba:640, mba:852, mba:307, mba:106, mba:1048,...",3,370,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:354,MY-mot,"Medulla, motor related",6,nsg:BrainRegion
4,http://api.brain-map.org/api/v2/data/Structure...,Class,Flocculus,413,FFFC91,1092,"[mba:10691, mba:10692, mba:10690]",3,1049,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:1073,FL,Flocculus,8,nsg:BrainRegion
5,http://api.brain-map.org/api/v2/data/Structure...,Class,"Flocculus, granular layer",None,ECE754,1095,NaN,3,10690,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:1049,FLgr,"Flocculus, granular layer",11,nsg:BrainRegion
6,http://api.brain-map.org/api/v2/data/Structure...,Class,"Flocculus, Purkinje layer",None,FFFC91,1094,NaN,3,10691,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:1049,FLpu,"Flocculus, Purkinje layer",11,nsg:BrainRegion
7,http://api.brain-map.org/api/v2/data/Structure...,Class,"Flocculus, molecular layer",None,FFFC91,1093,NaN,3,10692,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:1049,FLmo,"Flocculus, molecular layer",11,nsg:BrainRegion
8,http://api.brain-map.org/api/v2/data/Structure...,Class,"Superior olivary complex, medial part",437,FFAE6F,904,NaN,3,105,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:398,SOCm,"Superior olivary complex, medial part",9,nsg:BrainRegion
9,http://api.brain-map.org/api/v2/data/Structure...,Class,Superior olivary complex,332,FFAE6F,902,"[mba:122, mba:105, mba:114]",3,398,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:1132,SOC,Superior olivary complex,8,nsg:BrainRegion


## See all (transitive) ancestors of a type

In [21]:
# "ancestors" are inverse of the subclasses. 
# '^' means reverse/backward navigation of relations. 
# Remove the '*' for direct parents.
query = {"^subClassOf*":{"id":type_}} 
ancestors = forge.search(query, limit=1000, search_in_graph=False)

In [22]:
forge.as_dataframe(ancestors)

,id,type,label,isDefinedBy,altLabel,definition,subClassOf,prefLabel
0,http://www.w3.org/ns/prov#Entity,Class,Entity,http://bbp.epfl.ch/neurosciencegraph/ontologie...,NaN,NaN,NaN,NaN
1,https://neuroshapes.org/BrainRegion,Class,Brain Region,https://bbp.epfl.ch/ontologies/core/bmo,regional part of the brain,Anatomical divisons of the brain according to ...,bmo:BiologicalBrainComponent,NaN
2,https://bbp.epfl.ch/ontologies/core/bmo/Biolog...,Class,Biological Brain Component,http://bbp.epfl.ch/neurosciencegraph/ontologie...,NaN,NaN,prov:Entity,Biological Brain Component


## See (transitively) all related types through a given relation

In [23]:
ontology_type_of_interest = "http://api.brain-map.org/api/v2/data/Structure/39" #Anterior cingulate area, dorsal part
relation = "isPartOf"
query = {f"{relation}*":{"id":f"{ontology_type_of_interest}"}} #Remove the '*' for direct relationship.
related = forge.search(query, limit=10, search_in_graph=False)

In [24]:
forge.as_dataframe(related)

,id,type,label,atlas_id,color_hex_triplet,graph_order,hemisphere_id,identifier,isDefinedBy,isPartOf,notation,prefLabel,st_level,subClassOf,hasPart
0,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 5",975,40A666,229,3,1015,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:39,ACAd5,"Anterior cingulate area, dorsal part, layer 5",11,nsg:BrainRegion,NaN
1,http://api.brain-map.org/api/v2/data/Structure/39,Class,"Anterior cingulate area, dorsal part",4,40A666,226,3,39,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:31,ACAd,"Anterior cingulate area, dorsal part",9,nsg:BrainRegion,"[mba:1015, mba:919, mba:935, mba:211, mba:927]"
2,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 2/3",1016,40A666,228,3,211,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:39,ACAd2/3,"Anterior cingulate area, dorsal part, layer 2/3",11,nsg:BrainRegion,NaN
3,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 6a",963,40A666,230,3,919,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:39,ACAd6a,"Anterior cingulate area, dorsal part, layer 6a",11,nsg:BrainRegion,NaN
4,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 6b",964,40A666,231,3,927,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:39,ACAd6b,"Anterior cingulate area, dorsal part, layer 6b",11,nsg:BrainRegion,NaN
5,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 1",965,40A666,227,3,935,http://bbp.epfl.ch/neurosciencegraph/ontologie...,mba:39,ACAd1,"Anterior cingulate area, dorsal part, layer 1",11,nsg:BrainRegion,NaN


## Resolve a text to an ontology type

A [Resolver](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resolving) is used to link terms or a `Resource` to identifiers (URIs) in a knowledge graph thus addressing lexical variations
(merging of synonyms, aliases and acronyms) and disambiguating them.

In [25]:
from kgforge.core.commons.strategies import ResolvingStrategy
text = "cAD" #other example texts: L23_LBC, PV+, APDrop, cAD, somatosensory
limit=20

In [26]:
# other Search strategy can be ResolvingStrategy.BEST_MATCH, ResolvingStrategy.EXACT_MATCH
resolved_ontology_types = forge.resolve(text, scope="ontology", target="terms", 
                                        strategy=ResolvingStrategy.ALL_MATCHES,
                                        limit=limit)

In [27]:
forge.as_dataframe(resolved_ontology_types).head(100)

,id,type,label,isDefinedBy,notation,prefLabel,subClassOf
0,http://uri.interlex.org/base/ilx_0738207,Class,cAD,http://bbp.epfl.ch/neurosciencegraph/ontologie...,cAD,Continuous adapting electrical type,[http://bbp.epfl.ch/neurosciencegraph/ontologi...
1,http://uri.interlex.org/base/ilx_0738250,Class,cAD-noscltb,http://bbp.epfl.ch/neurosciencegraph/ontologie...,cAD-noscltb,Continuous adapting non-oscillatory low-thresh...,[http://bbp.epfl.ch/neurosciencegraph/ontologi...
2,http://uri.interlex.org/base/ilx_0738255,Class,cAD-ltb,http://bbp.epfl.ch/neurosciencegraph/ontologie...,cAD-ltb,Continuous adapting low-threshold bursting ele...,[http://bbp.epfl.ch/neurosciencegraph/ontologi...
3,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 6a",http://bbp.epfl.ch/neurosciencegraph/ontologie...,ACAd6a,"Anterior cingulate area, dorsal part, layer 6a",nsg:BrainRegion
4,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 6b",http://bbp.epfl.ch/neurosciencegraph/ontologie...,ACAd6b,"Anterior cingulate area, dorsal part, layer 6b",nsg:BrainRegion
5,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 1",http://bbp.epfl.ch/neurosciencegraph/ontologie...,ACAd1,"Anterior cingulate area, dorsal part, layer 1",nsg:BrainRegion
6,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 5",http://bbp.epfl.ch/neurosciencegraph/ontologie...,ACAd5,"Anterior cingulate area, dorsal part, layer 5",nsg:BrainRegion
7,https://bbp.epfl.ch/ontologies/core/bmo/Taudec...,Class,Tau decay CaDynamics DC0,../../ontologies/core/bmo,NaN,NaN,bmo:CalciumDynamics
8,http://api.brain-map.org/api/v2/data/Structure...,Class,"Anterior cingulate area, dorsal part, layer 2/3",http://bbp.epfl.ch/neurosciencegraph/ontologie...,ACAd2/3,"Anterior cingulate area, dorsal part, layer 2/3",nsg:BrainRegion
9,https://bbp.epfl.ch/ontologies/core/bmo/GammaC...,Class,Gamma CaDynamics DC0,../../ontologies/core/bmo,NaN,NaN,bmo:CalciumDynamics


### BEST_MATCH resolving

In [28]:
resolved_ontology_types = forge.resolve(text, scope="ontology", target="terms",
                                        strategy=ResolvingStrategy.BEST_MATCH,
                                        limit=limit)

In [29]:
print(resolved_ontology_types)

{
    id: http://api.brain-map.org/api/v2/data/Structure/1015
    type: Class
    label: Anterior cingulate area, dorsal part, layer 5
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/mba
    notation: ACAd5
    prefLabel: Anterior cingulate area, dorsal part, layer 5
    subClassOf: nsg:BrainRegion
}


### EXACT_MATCH resolving

In [30]:
resolved_ontology_types = forge.resolve(text, scope="ontology", target="terms", 
                                        strategy=ResolvingStrategy.EXACT_MATCH,
                                        limit=limit)


In [31]:
print(resolved_ontology_types)

{
    id: http://uri.interlex.org/base/ilx_0738207
    type: Class
    label: cAD
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/etypes
    notation: cAD
    prefLabel: Continuous adapting electrical type
    subClassOf:
    [
        http://bbp.epfl.ch/neurosciencegraph/ontologies/celltypes/etypes/NeocortexEType
        http://bbp.epfl.ch/neurosciencegraph/ontologies/celltypes/etypes/ThalamusEType
        http://bbp.epfl.ch/neurosciencegraph/ontologies/etypes/NeocortexEType
        http://bbp.epfl.ch/neurosciencegraph/ontologies/etypes/ThalamusEType
    ]
}
